In [8]:
# import torch
import torch
import os
from torch import cat, Tensor
from torch.nn import Module
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset, ConcatDataset, TensorDataset
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torchvision import datasets, transforms
import torch.optim.lr_scheduler # ?
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, CenterCrop, RandomHorizontalFlip, Resize
from torchvision.transforms.functional import center_crop
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.utils import save_image
from torchvision.transforms.functional import pil_to_tensor
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import os

stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

transform = transform_train = Compose([
    # Resize(224),
    # Resize(384),
    # RandomHorizontalFlip(),
    ToTensor(),
    # Normalize(*stats,inplace=True)
])

# Load the CIFAR-100 training set
trainset = torchvision.datasets.CIFAR100(root='data', train=True,download=True, transform=transform)

name_list = trainset.classes

import json

def read_json(json_path):
# Step 1: Open the JSON file
    with open(json_path, 'r') as file:
    # Step 2: Load the JSON data
        data = json.load(file)
    return data

# name_dict = read_json('/scratch/local/ssd/enbo/saved_data/imagenet_label_name.json')
# name_list = [item for key, item in name_dict.items()]
order_list = [[36, 0, 54, 5, 20],
 [22, 45, 13, 83, 19],
 [26, 73, 16, 62, 33],
 [34, 98, 24, 74, 53],
 [10, 94, 51, 4, 32],
 [38, 81, 50, 40, 41],
 [30, 89, 69, 64, 21],
 [84, 14, 88, 49, 68],
 [6, 80, 57, 65, 46],
 [9, 91, 48, 72, 31],
 [76, 7, 47, 8, 1],
 [61, 75, 63, 18, 86],
 [59, 70, 43, 85, 95],
 [27, 93, 35, 25, 82],
 [44, 56, 67, 66, 37],
 [60, 11, 2, 78, 52],
 [97, 39, 55, 3, 99],
 [29, 71, 23, 28, 90],
 [87, 15, 92, 17, 77],
 [12, 42, 96, 79, 58]]

syn_classes = [order[3:] for order in order_list]
syn_classes = [item for lists in syn_classes for item in lists]

# syn_classes = [5, 20, 83, 19, 62, 33, 74, 53, 4, 32, 40, 41, 64, 21, 49, 68, 65, 46, 72, 31, 8, 1, 18, 86, 85, 95, 25, 82, 66, 37, 78, 52, 3, 99, 28, 90, 17, 77, 79, 58]
real_classes = list(set([i for i in range(100)]) - set(syn_classes))
print(len(real_classes))

Files already downloaded and verified
60


In [9]:
import torch
torch.cuda.set_device(6)
if torch.cuda.is_available():
    current_gpu = torch.cuda.current_device()
    print(f"Current default GPU index: {current_gpu}")
    print(f"Current default GPU name: {torch.cuda.get_device_name(current_gpu)}")
else:
    print("No GPUs available.")

Current default GPU index: 6
Current default GPU name: NVIDIA A40


In [10]:
integer_to_name = {i: name_list[i] for i in range(100)}
name_to_integer = {name_list[i]: i for i in range(100)}

In [11]:
# Importing necessary libraries
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Initializing the Sentence Transformer model using BERT with mean-tokens pooling
model = SentenceTransformer('bert-base-nli-mean-tokens')
# model = SentenceTransformer("all-mpnet-base-v2")


In [12]:
real_class_names = [name_list[i] for i in real_classes]
syn_class_names = [name_list[i] for i in syn_classes]

In [13]:
len(syn_class_names)

40

In [14]:
embeddings_40 = model.encode(syn_class_names)
embeddings_60 = model.encode(real_class_names)

# Calculating the cosine similarity matrix between the two sets of embeddings
similarity_matrix = cosine_similarity(embeddings_40, embeddings_60)

# Finding the most similar class from the 60 for each of the 40 classes
most_similar_classes = []
dict_40to60_name = {}
for index, similarities in enumerate(similarity_matrix):
    # Get the index of the maximum similarity score in this row
    max_index = similarities.argmax()
    # Append the corresponding class from class_names_60
    most_similar_classes.append(real_class_names[max_index])
    dict_40to60_name[syn_class_names[index]] = real_class_names[max_index]
    # Optionally print each pair
    print(f"Most similar to {syn_class_names[index]} is {real_class_names[max_index]} with a similarity score of {similarities[max_index]:.2f}")


Most similar to bed is television with a similarity score of 0.64
Most similar to chair is table with a similarity score of 0.81
Most similar to sweet_pepper is mushroom with a similarity score of 0.63
Most similar to cattle is tractor with a similarity score of 0.72
Most similar to poppy is rose with a similarity score of 0.79
Most similar to forest is pine_tree with a similarity score of 0.77
Most similar to shrew is porcupine with a similarity score of 0.75
Most similar to orange is can with a similarity score of 0.73
Most similar to beaver is otter with a similarity score of 0.73
Most similar to flatfish is crab with a similarity score of 0.74
Most similar to lamp is bottle with a similarity score of 0.78
Most similar to lawn_mower is tractor with a similarity score of 0.73
Most similar to possum is cockroach with a similarity score of 0.78
Most similar to chimpanzee is dinosaur with a similarity score of 0.65
Most similar to mountain is bridge with a similarity score of 0.63
Most 

In [15]:
print(dict_40to60_name)

{'bed': 'television', 'chair': 'table', 'sweet_pepper': 'mushroom', 'cattle': 'tractor', 'poppy': 'rose', 'forest': 'pine_tree', 'shrew': 'porcupine', 'orange': 'can', 'beaver': 'otter', 'flatfish': 'crab', 'lamp': 'bottle', 'lawn_mower': 'tractor', 'possum': 'cockroach', 'chimpanzee': 'dinosaur', 'mountain': 'bridge', 'road': 'bridge', 'rabbit': 'squirrel', 'man': 'boy', 'seal': 'bottle', 'elephant': 'leopard', 'bicycle': 'motorcycle', 'aquarium_fish': 'crab', 'caterpillar': 'cockroach', 'telephone': 'clock', 'tank': 'bottle', 'whale': 'shark', 'couch': 'television', 'sunflower': 'tulip', 'raccoon': 'tiger', 'house': 'wardrobe', 'snake': 'lizard', 'oak_tree': 'maple_tree', 'bear': 'wolf', 'worm': 'skunk', 'cup': 'bottle', 'train': 'streetcar', 'castle': 'clock', 'snail': 'crab', 'spider': 'squirrel', 'pickup_truck': 'tractor'}


In [16]:
dict_40to60_id = {}
for key in dict_40to60_name:
    dict_40to60_id[name_to_integer[key]] = name_to_integer[dict_40to60_name[key]]
    

In [17]:
print(dict_40to60_id)

{5: 87, 20: 84, 83: 51, 19: 89, 62: 70, 33: 59, 74: 63, 53: 16, 4: 55, 32: 26, 40: 9, 41: 89, 64: 24, 21: 29, 49: 12, 68: 12, 65: 80, 46: 11, 72: 9, 31: 42, 8: 48, 1: 26, 18: 24, 86: 22, 85: 9, 95: 73, 25: 87, 82: 92, 66: 88, 37: 94, 78: 44, 52: 47, 3: 97, 99: 75, 28: 9, 90: 81, 17: 22, 77: 26, 79: 80, 58: 89}


In [18]:
with open('/storage3/enbo/saved_data/cifar100_dict_40synfrom60real_id.json', 'w') as json_file:
    json.dump(dict_40to60_id, json_file, indent=4)

In [19]:
with open('/storage3/enbo/saved_data/cifar100_dict_40synfrom60real_name.json', 'w') as json_file:
    json.dump(dict_40to60_name, json_file, indent=4)

In [25]:
from avalanche.benchmarks import SplitMNIST, SplitCIFAR100

# sythnthesis classes
benchmark = SplitCIFAR100(n_experiences=20,
                          seed = 41,             
                          )

orders = benchmark.classes_order
order_list = [orders[x:x+5] for x in range(0, len(orders), 5)]

order_sample = [order[3:] for order in order_list]
classname_list = []
label_list = []
classname_list_sep = []
for order_l in order_sample:
    label_list.append(order_l)
    cur_classname = [integer_to_name[i] for i in order_l]
    classname_list.append(cur_classname)
classname_list_sep = [item for lists in classname_list for item in lists]
label_list_sep = [item for lists in label_list for item in lists]
print(label_list_sep)

Files already downloaded and verified
Files already downloaded and verified
[5, 20, 83, 19, 62, 33, 74, 53, 4, 32, 40, 41, 64, 21, 49, 68, 65, 46, 72, 31, 8, 1, 18, 86, 85, 95, 25, 82, 66, 37, 78, 52, 3, 99, 28, 90, 17, 77, 79, 58]


In [27]:
order_list

[[36, 0, 54, 5, 20],
 [22, 45, 13, 83, 19],
 [26, 73, 16, 62, 33],
 [34, 98, 24, 74, 53],
 [10, 94, 51, 4, 32],
 [38, 81, 50, 40, 41],
 [30, 89, 69, 64, 21],
 [84, 14, 88, 49, 68],
 [6, 80, 57, 65, 46],
 [9, 91, 48, 72, 31],
 [76, 7, 47, 8, 1],
 [61, 75, 63, 18, 86],
 [59, 70, 43, 85, 95],
 [27, 93, 35, 25, 82],
 [44, 56, 67, 66, 37],
 [60, 11, 2, 78, 52],
 [97, 39, 55, 3, 99],
 [29, 71, 23, 28, 90],
 [87, 15, 92, 17, 77],
 [12, 42, 96, 79, 58]]

In [29]:
order_list_names = []
for l in order_list:
    names = [integer_to_name[item] for item in l]
    order_list_names.append(names)
    

In [31]:
print(order_list_names)

[['hamster', 'apple', 'orchid', 'bed', 'chair'], ['clock', 'lobster', 'bus', 'sweet_pepper', 'cattle'], ['crab', 'shark', 'can', 'poppy', 'forest'], ['fox', 'woman', 'cockroach', 'shrew', 'orange'], ['bowl', 'wardrobe', 'mushroom', 'beaver', 'flatfish'], ['kangaroo', 'streetcar', 'mouse', 'lamp', 'lawn_mower'], ['dolphin', 'tractor', 'rocket', 'possum', 'chimpanzee'], ['table', 'butterfly', 'tiger', 'mountain', 'road'], ['bee', 'squirrel', 'pear', 'rabbit', 'man'], ['bottle', 'trout', 'motorcycle', 'seal', 'elephant'], ['skyscraper', 'beetle', 'maple_tree', 'bicycle', 'aquarium_fish'], ['plate', 'skunk', 'porcupine', 'caterpillar', 'telephone'], ['pine_tree', 'rose', 'lion', 'tank', 'whale'], ['crocodile', 'turtle', 'girl', 'couch', 'sunflower'], ['lizard', 'palm_tree', 'ray', 'raccoon', 'house'], ['plain', 'boy', 'baby', 'snake', 'oak_tree'], ['wolf', 'keyboard', 'otter', 'bear', 'worm'], ['dinosaur', 'sea', 'cloud', 'cup', 'train'], ['television', 'camel', 'tulip', 'castle', 'snail']

In [36]:
dict_syn2real_id_withinexperience = {}
dict_syn2real_name_withinexperience = {}
most_sim_all = []
for l in order_list_names:
    l_real, l_syn = l[:3], l[3:]
    embedding_real, embedding_syn = model.encode(l_real), model.encode(l_syn)
    sim_matrix = cosine_similarity(embedding_syn, embedding_real)
    most_sim = []
    for index, similarities in enumerate(sim_matrix):
    # Get the index of the maximum similarity score in this row
        max_index = similarities.argmax()
        most_sim.append(l[max_index])
    most_sim_all.append(most_sim)
        
    

In [37]:
most_sim_all

[['hamster', 'orchid'],
 ['lobster', 'bus'],
 ['can', 'can'],
 ['fox', 'fox'],
 ['mushroom', 'mushroom'],
 ['streetcar', 'streetcar'],
 ['tractor', 'dolphin'],
 ['table', 'table'],
 ['squirrel', 'bee'],
 ['bottle', 'bottle'],
 ['skyscraper', 'beetle'],
 ['porcupine', 'plate'],
 ['rose', 'lion'],
 ['girl', 'girl'],
 ['lizard', 'ray'],
 ['baby', 'baby'],
 ['wolf', 'wolf'],
 ['cloud', 'cloud'],
 ['television', 'camel'],
 ['leopard', 'bridge']]

In [38]:
most_sim_all_id = []
for l in most_sim_all:
    sim_id = [name_to_integer[item] for item in l]
    most_sim_all_id.append(sim_id)
    

In [39]:
most_sim_all_id

[[36, 54],
 [45, 13],
 [16, 16],
 [34, 34],
 [51, 51],
 [81, 81],
 [89, 30],
 [84, 84],
 [80, 6],
 [9, 9],
 [76, 7],
 [63, 61],
 [70, 43],
 [35, 35],
 [44, 67],
 [2, 2],
 [97, 97],
 [23, 23],
 [87, 15],
 [42, 12]]

In [4]:
order_list = [[36, 0, 54, 5, 20],
 [22, 45, 13, 83, 19],
 [26, 73, 16, 62, 33],
 [34, 98, 24, 74, 53],
 [10, 94, 51, 4, 32],
 [38, 81, 50, 40, 41],
 [30, 89, 69, 64, 21],
 [84, 14, 88, 49, 68],
 [6, 80, 57, 65, 46],
 [9, 91, 48, 72, 31],
 [76, 7, 47, 8, 1],
 [61, 75, 63, 18, 86],
 [59, 70, 43, 85, 95],
 [27, 93, 35, 25, 82],
 [44, 56, 67, 66, 37],
 [60, 11, 2, 78, 52],
 [97, 39, 55, 3, 99],
 [29, 71, 23, 28, 90],
 [87, 15, 92, 17, 77],
 [12, 42, 96, 79, 58]]

most_sim_id = [[36, 54],
 [45, 13],
 [16, 16],
 [34, 34],
 [51, 51],
 [81, 81],
 [89, 30],
 [84, 84],
 [80, 6],
 [9, 9],
 [76, 7],
 [63, 61],
 [70, 43],
 [35, 35],
 [44, 67],
 [2, 2],
 [97, 97],
 [23, 23],
 [87, 15],
 [42, 12]]
order_list_names = [['hamster', 'apple', 'orchid', 'bed', 'chair'], ['clock', 'lobster', 'bus', 'sweet_pepper', 'cattle'], ['crab', 'shark', 'can', 'poppy', 'forest'], ['fox', 'woman', 'cockroach', 'shrew', 'orange'], ['bowl', 'wardrobe', 'mushroom', 'beaver', 'flatfish'], ['kangaroo', 'streetcar', 'mouse', 'lamp', 'lawn_mower'], ['dolphin', 'tractor', 'rocket', 'possum', 'chimpanzee'], ['table', 'butterfly', 'tiger', 'mountain', 'road'], ['bee', 'squirrel', 'pear', 'rabbit', 'man'], ['bottle', 'trout', 'motorcycle', 'seal', 'elephant'], ['skyscraper', 'beetle', 'maple_tree', 'bicycle', 'aquarium_fish'], ['plate', 'skunk', 'porcupine', 'caterpillar', 'telephone'], ['pine_tree', 'rose', 'lion', 'tank', 'whale'], ['crocodile', 'turtle', 'girl', 'couch', 'sunflower'], ['lizard', 'palm_tree', 'ray', 'raccoon', 'house'], ['plain', 'boy', 'baby', 'snake', 'oak_tree'], ['wolf', 'keyboard', 'otter', 'bear', 'worm'], ['dinosaur', 'sea', 'cloud', 'cup', 'train'], ['television', 'camel', 'tulip', 'castle', 'snail'], ['bridge', 'leopard', 'willow_tree', 'spider', 'pickup_truck']]
sim_names = [['hamster', 'orchid'],
 ['lobster', 'bus'],
 ['can', 'can'],
 ['fox', 'fox'],
 ['mushroom', 'mushroom'],
 ['streetcar', 'streetcar'],
 ['tractor', 'dolphin'],
 ['table', 'table'],
 ['squirrel', 'bee'],
 ['bottle', 'bottle'],
 ['skyscraper', 'beetle'],
 ['porcupine', 'plate'],
 ['rose', 'lion'],
 ['girl', 'girl'],
 ['lizard', 'ray'],
 ['baby', 'baby'],
 ['wolf', 'wolf'],
 ['cloud', 'cloud'],
 ['television', 'camel'],
 ['leopard', 'bridge']]

In [5]:
expsim_dict = {}
syn_classes = [item[3:] for item in order_list_names]
for i, l in enumerate(syn_classes):
    for j, item in enumerate(l):
        expsim_dict[item] = sim_names[i][j]

In [6]:
print(expsim_dict)

{'bed': 'hamster', 'chair': 'orchid', 'sweet_pepper': 'lobster', 'cattle': 'bus', 'poppy': 'can', 'forest': 'can', 'shrew': 'fox', 'orange': 'fox', 'beaver': 'mushroom', 'flatfish': 'mushroom', 'lamp': 'streetcar', 'lawn_mower': 'streetcar', 'possum': 'tractor', 'chimpanzee': 'dolphin', 'mountain': 'table', 'road': 'table', 'rabbit': 'squirrel', 'man': 'bee', 'seal': 'bottle', 'elephant': 'bottle', 'bicycle': 'skyscraper', 'aquarium_fish': 'beetle', 'caterpillar': 'porcupine', 'telephone': 'plate', 'tank': 'rose', 'whale': 'lion', 'couch': 'girl', 'sunflower': 'girl', 'raccoon': 'lizard', 'house': 'ray', 'snake': 'baby', 'oak_tree': 'baby', 'bear': 'wolf', 'worm': 'wolf', 'cup': 'cloud', 'train': 'cloud', 'castle': 'television', 'snail': 'camel', 'spider': 'leopard', 'pickup_truck': 'bridge'}
